## Importing Necessary Libraries

In [9]:
# pip install pynvml

In [10]:
import warnings
warnings.filterwarnings('ignore')
import os, evaluate, torch, accelerate, nltk, re, datetime
import pandas as pd
from collections import Counter
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForTokenClassification
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
import seaborn as sns
from pynvml import *
from numba import cuda
import matplotlib.pyplot as plt

## Get Current Working Directory

In [11]:
my_dir = os.getcwd() + '/'
print(f"CWD: {my_dir}")

CWD: /content/


## GPU Configuration

<p> Reset Cuda Memory</p>

In [13]:
# device = cuda.get_current_device()
# device.reset()
# with torch.no_grad():
#     torch.cuda.empty_cache()

<p>Check GPU Memory</p>

In [15]:
# nvmlInit()
# h = nvmlDeviceGetHandleByIndex(0)
# info = nvmlDeviceGetMemoryInfo(h)
# print(f'Total    : {info.total//1024**2} MB')
# print(f'Free     : {info.free//1024**2} MB')
# print(f'Used     : {info.used//1024**2} MB')

### Get Cuda Device

In [17]:
devices = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
devices

device(type='cpu')

## Load Modela nd Tokenizer

In [18]:
tokenizer = AutoTokenizer.from_pretrained("Clinical-AI-Apollo/Medical-NER")
pre_trained_model = AutoModelForTokenClassification.from_pretrained("Clinical-AI-Apollo/Medical-NER")

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/736M [00:00<?, ?B/s]

In [19]:
pipe = pipeline("token-classification", tokenizer = tokenizer , model = pre_trained_model)

In [23]:
hpi1 = """
Patient was referred to Physical Therapy due to change in functional performance of: walking,
transfers, bed mobility, and wheelchair mobility
When: Approximately 02-19-21
Factors contributing to functional change: recent hospitalization, change in previous condition, and change in balance.
Additional comments including complication, comorbidity, or safety issue warranting skilled care: Pt admitted to hospital 3
weeks ago w/ a kidney infection secondary to a kidney stone. He presents w/ signficant deconditioning and gait disturbance,
which limit his ability to safely transfer and ambulate within his SLC.
Recomendation should be Panadol, Brufene, Paracetamol and one tble spoon of pink salt.
"""

hpi2 = """
Patient Summary:
Reason for Referral:
Patient was referred to Occupational Therapy due to changes in functional performance of: self-care activities, fine motor skills, and upper extremity strength.
When:
Approximately 05-12-22
Factors Contributing to Functional Change:
Recent stroke
Change in cognitive status
Decreased coordination
Additional Comments Including Complication, Comorbidity, or Safety Issue Warranting Skilled Care:
Patient experienced a stroke 2 months ago, leading to significant impairments in motor function and cognitive processing. He presents with decreased upper extremity strength and fine motor coordination, impacting his ability to perform daily self-care tasks such as dressing and feeding. There is a noted risk of falls due to impaired balance and coordination.
Recommendations:
Pain Management: Tylenol (Acetaminophen) 500mg as needed
Anti-inflammatory: Ibuprofen 400mg every 6 hours as needed
Muscle Relaxant: Baclofen 10mg three times daily
Dietary Supplement: One tablespoon of Epsom salt dissolved in warm water daily
"""

In [24]:
pred = pipe(hpi2)

In [25]:
print("Index | Word     | Entity                     | Score")
print("-" * 75)
for item in pred:
    print(f"{item['index']:5} | {item['word']:8} | {item['entity']:20} | {item['score']:.5f}")


Index | Word     | Entity                     | Score
---------------------------------------------------------------------------
   10 | ▁referred | B-CLINICAL_EVENT     | 0.80590
   12 | ▁Occupational | B-NONBIOLOGICAL_LOCATION | 0.37753
   13 | ▁Therapy | I-NONBIOLOGICAL_LOCATION | 0.41005
   19 | ▁performance | I-DIAGNOSTIC_PROCEDURE | 0.50059
   22 | ▁self    | B-DIAGNOSTIC_PROCEDURE | 0.41890
   23 | -        | I-DIAGNOSTIC_PROCEDURE | 0.53196
   24 | care     | I-DIAGNOSTIC_PROCEDURE | 0.70614
   27 | ▁fine    | B-DIAGNOSTIC_PROCEDURE | 0.91354
   28 | ▁motor   | I-DIAGNOSTIC_PROCEDURE | 0.88432
   29 | ▁skills  | I-DIAGNOSTIC_PROCEDURE | 0.63265
   32 | ▁upper   | B-BIOLOGICAL_STRUCTURE | 0.40217
   33 | ▁extremity | I-BIOLOGICAL_STRUCTURE | 0.41820
   34 | ▁strength | I-DIAGNOSTIC_PROCEDURE | 0.48439
   39 | ▁05      | B-DATE               | 0.42404
   40 | -        | I-DATE               | 0.67954
   41 | 12       | I-DATE               | 0.73661
   42 | -        | I-DATE    